# Wstęp

Metody uczenia maszynowego możemy podzielić na dwie główne kategorie (pomijając uczenie ze wzmocnieniem): nadzorowane i nienadzorowane. Uczenie **nadzorowane** (ang. *supervised*) to jest uczenie z dostępnymi etykietami dla danych wejściowych. Na parach danych uczących $dataset= \{(x_0,y_0), (x_1,y_1), \ldots, (x_n,y_n)\}$ model ma za zadanie nauczyć się funkcji $f: X \rightarrow Y$. Z kolei modele uczone w sposób **nienadzorowany** (ang. *unsupervised*) wykorzystują podczas trenowania dane nieetykietowane tzn. nie znamy $y$ z pary $(x, y)$.

Dość częstą sytuacją, z jaką mamy do czynienia, jest posiadanie małego podziobioru danych etykietowanych i dużego nieetykietowanych. Często annotacja danych wymaga ingerencji człowieka - ktoś musi określić co jest na obrazku, ktoś musi powiedzieć czy dane słowo jest rzeczownkiem czy czasownikiem itd.

Jeżeli mamy dane etykietowane do zadania uczenia nadzorowanego (np. klasyfikacja obrazka), ale także dużą ilość danych nieetykietowanych, to możemy wtedy zastosować techniki **uczenia częściowo nadzorowanego** (ang. *semi-supervised learning*). Te techniki najczęściej uczą się funkcji $f: X \rightarrow Y$, ale jednocześnie są w stanie wykorzystać informacje z danych nieetykietowanych do poprawienia działania modelu.

## Cel ćwiczenia

Celem ćwiczenia jest nauczenie modelu z wykorzystaniem danych etykietowanych i nieetykietowanych ze zbioru STL10 z użyciem metody [Bootstrap your own latent](https://arxiv.org/abs/2006.07733).

Metoda ta jest relatywnie "lekka" obliczeniowo, a także dość prosta do zrozumienia i zaimplementowania, dlatego też na niej się skupimy na tych laboratoriach.

# Zbiór STL10

Zbiór STL10 to zbiór stworzony i udostępniony przez Stanford [[strona]](https://ai.stanford.edu/~acoates/stl10/) [[papier]](https://cs.stanford.edu/~acoates/papers/coatesleeng_aistats_2011.pdf) a inspirowany przez CIFAR-10. Obrazy zostały pozyskane z [ImageNet](https://image-net.org/). Szczegóły można doczytać na ich stronie. To co jest ważne to to, że autorzy zbioru dostarczają predefiniowany plan eksperymentalny, żeby móc porównywać łatwo wyniki eksperymentów. Nie będziemy go tutaj stosować z uwagi na jego czasochłonność (10 foldów), ale warto pamiętać o tym, że często są z góry ustalone sposoby walidacji zaprojetowanych przez nas algorytmów na określonych zbiorach referencyjnych.

Korzystając z `torchvision.datasets` ***załaduj*** 3 podziały zbioru danych STL10: `train`, `test`, `unlabeled` oraz utwórz z nich instancje klasy `DataLoader`. Korzystając z Google Colab rozważ użycie Google Drive do przechowyania zbioru w calu zaoszczędzenia czasu na wielokrotne pobieranie.

In [12]:
import torchvision.datasets as datasets
import torch
from torch.utils.data import DataLoader
import os
from google.colab import drive
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


drive.mount("/content/drive/")

path = '/content/drive/MyDrive/lab09/data/'
os.makedirs(path, exist_ok=True)

transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_data = datasets.STL10(path, split='train', download=True, transform=transform)
test_data = datasets.STL10(path, split='test', download=True, transform=transform)
unlabeled_data = datasets.STL10(path, split='unlabeled', download=True, transform=transform)

# dataset_length = len(unlabeled_data)
# train_length = int(0.78 * dataset_length)
# val_length = dataset_length - train_length

# # Split the dataset
# unlabeled_data, val_dataset = torch.utils.data.random_split(unlabeled_data, [train_length, val_length])



batch_size = 32  # Możesz dostosować rozmiar partii do swoich potrzeb

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(unlabeled_data, batch_size=256, shuffle=True)





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Uczenie nadzorowane

Żeby porównać czy metoda BYOL przynosi nam jakieś korzyści musimy wyznaczyć wartość bazową metryk(i) jakości, których będziemu używać (np. dokładność).

***Zaimplementuj*** wybraną metodę uczenia nadzorowanego na danych `train` z STL10. Możesz wykorzystać predefiniowane architektury w `torchvision.models` oraz kody źródłowe z poprzednich list.

In [13]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


class SimpleCNNEncoder(torch.nn.Module):
    def __init__(self, repr=80):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Sequential(
            nn.Linear(128*4*4*9, 512),  # Assuming input image size is 32x32
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, repr)
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x



class MyLinear(torch.nn.Module):
  def __init__(self, encoder):
    super().__init__()
    self.encoder = encoder

    self.fc = nn.Sequential(
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 10),

    )

    # for param in self.encoder.parameters():
    #     param.requires_grad = False

  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

In [14]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from typing import Tuple
from tqdm import tqdm


def count_correct(
    y_pred: torch.Tensor, y_true: torch.Tensor
) -> torch.Tensor:
    preds = torch.argmax(y_pred, dim=1)
    return (preds == y_true).float().sum()

def validate(
    model: nn.Module,
    loss_fn: torch.nn.CrossEntropyLoss,
    dataloader: DataLoader
) -> Tuple[torch.Tensor, torch.Tensor]:
    loss = 0
    correct = 0
    all = 0
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch.cuda())
        all += len(y_pred)
        loss += loss_fn(y_pred, y_batch.cuda()).sum()
        correct += count_correct(y_pred, y_batch.cuda())
    return loss / all, correct / all

def fit(
    model: nn.Module, optimiser: optim.Optimizer,
    loss_fn: torch.nn.CrossEntropyLoss,
    train_dl: DataLoader,
    val_dl: DataLoader,
    epochs: int,
    print_metrics: bool = False,
    name: str ='',
    patience: int = 3,
):
    last_better_epoch = 0
    low_val_loss = float('inf')
    low_acc_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in tqdm(train_dl, desc=f"Epoch {epoch}"):
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch.long())

            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

        model.eval()
        with torch.no_grad():
            train_loss, train_acc = validate(model, loss_fn, train_dl)
            val_loss, val_acc = validate(model, loss_fn, val_dl)

            if val_loss < low_val_loss:
                low_val_loss = val_loss
                low_acc_loss = val_acc
                last_better_epoch = epoch

            if patience is not None and epoch - last_better_epoch > patience:
                print(f"Early stopping on epoch {epoch}")
                break

            # writer.add_scalars(
            #     main_tag='loss',
            #     tag_scalar_dict={
            #         f'train_{name}': train_loss,
            #         f'dev_{name}': val_loss
            #     },
            #     global_step=epoch+1
            # )

            # writer.add_scalars(
            #     main_tag=f'acc',
            #     tag_scalar_dict={
            #         f'train_{name}': train_acc,
            #         f'dev_{name}': val_acc
            #     },
            #     global_step=epoch+1
            # )
            if print_metrics:
                print(
                    f"Epoch {name} {epoch}: "
                    f"train loss = {train_loss:.3f} (acc: {train_acc:.3f}), "
                    f"validation loss = {val_loss:.3f} (acc: {val_acc:.3f})"
                )

    return low_val_loss, low_acc_loss

In [15]:
model = MyLinear(SimpleCNNEncoder(repr=256)).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='vgg19')
print(result)

Epoch 0: 100%|██████████| 157/157 [00:03<00:00, 45.35it/s]


Epoch vgg19 0: train loss = 0.050 (acc: 0.394), validation loss = 0.050 (acc: 0.372)


Epoch 1: 100%|██████████| 157/157 [00:03<00:00, 46.70it/s]


Epoch vgg19 1: train loss = 0.043 (acc: 0.455), validation loss = 0.046 (acc: 0.419)


Epoch 2: 100%|██████████| 157/157 [00:03<00:00, 46.43it/s]


...

Epoch vgg19 5: train loss = 0.020 (acc: 0.770), validation loss = 0.041 (acc: 0.566)


Epoch 6: 100%|██████████| 157/157 [00:03<00:00, 46.23it/s]


Epoch vgg19 6: train loss = 0.013 (acc: 0.870), validation loss = 0.041 (acc: 0.588)


Epoch 7: 100%|██████████| 157/157 [00:03<00:00, 45.24it/s]


Early stopping on epoch 7
(tensor(0.0376, device='cuda:0'), tensor(0.5590, device='cuda:0'))


# Bootstrap your own latent

Metoda [Bootstrap your own latent](https://arxiv.org/abs/2006.07733) jest opisana w rodziale 3.1 papieru a także w dodatku A. Składa się z dwóch etapów:


1.   uczenia samonadzorowanego (ang. *self-supervised*)
2.   douczania nadzorowanego (ang. *fine-tuning*)

## Uczenie samonadzorowane

Architektura do nauczania samonadzorowanego składa się z dwóch sieci: (1) *online* i (2) *target*. W uproszczeniu cała architektura działa tak:


1.   Dla obrazka $x$ wygeneruj dwie różne augmentacje $v$ i $v'$ za pomocą funkcji $t$ i $t'$.
2.   Widok $v$ przekazujemy do sieci *online*, a $v'$ do *target*.
3.   Następnie widoki przekształacamy za pomocą sieci do uczenia reprezentacji (np. resnet18 lub resnet50) do reprezentacji $y_\theta$ i $y'_\xi$.
4.   Potem dokonujemy projekcji tych reprezentacji w celu zmniejszenia wymiarowości (np. za pomocą sieci MLP).
5.   Na sieci online dokonujmey dodatkowo predykcji pseudo-etykiety (ang. *pseudolabel*)
6.   Wyliczamy fukncję kosztu: MSE z wyjścia predyktora sieci *online* oraz wyjścia projekcji sieci *target* "przepuszczonej" przez predyktor sieci *online* **bez propagacji wstecznej** (*vide Algorithm 1* z papieru).
7.   Dokonujemy wstecznej propagacji **tylko** po sieci *online*.
8.   Aktualizujemy wagi sieci *target* sumując w ważony sposób wagi obu sieci $\xi = \tau\xi + (1 - \tau)\theta$ ($\tau$ jest hiperprametrem) - jest to ruchoma średnia wykładnicza (ang. *moving exponential average*).

Po zakończeniu procesu uczenia samonadzorowanego zostawiamy do douczania sieć kodera *online* $f_\theta$. Cała sieć *target* oraz warstwy do projekcji i predykcji w sieci *online* są "do wyrzucenia".

### Augmentacja

Dodatek B publikacji opisuje augmentacje zastosowane w metodzie BYOL. Zwróć uwagę na tabelę 6 w publikacji. `torchvision.transforms.RandomApply` może być pomocne.

***Zaimeplementuj*** augmentację $\tau$ i $\tau'$.


In [4]:
import random
from torch import nn
import torch
from torchvision import transforms as T


def get_default_aug() -> nn.Module:
    return torch.nn.Sequential(
        RandomApply(T.RandomRotation(degrees=(10, 60)), p=0.2),
        T.RandomHorizontalFlip(),
        RandomApply(T.GaussianBlur((3, 3), (1.0, 2.0)), p=0.2),
    )


class RandomApply(nn.Module):
    def __init__(self, fn: nn.Module, p: float):
        super().__init__()
        self.fn = fn
        self.p = p

    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)


### Implementacja uczenia samonadzorowanego

***Zaprogramuj*** proces uczenia samonadzorowanego na danych nieetykietowanych ze zbioru STL10.

Wskazówki do realizacji polecenia:

1. Proces uczenia może trwać bardzo długo dlatego zaleca się zastsowanie wczesnego zatrzymania lub uczenia przez tylko jedną epokę. Mimo wszystko powinno się dać osiągnąć poprawę w uczeniu nadzorowanym wykorzystując tylko zasoby z Google Colab.
2. Dobrze jest pominąć walidację na zbiorze treningowym i robić ją tylko na zbiorze walidacyjnym - zbiór treningowy jest ogromny i w związku z tym narzut czasowy na walidację też będzie duży.
3. Walidację modelu można przeprowadzić na zbiorze `train` lub całkowicie ją pominąć, jeżeli uczymy na stałej ilości epok.
4. Rozważ zastosowanie tylko jednej augmentacji - augmentacja $\tau'$ jest bardziej czasochłonna niż $\tau$.
5. Poniżej jest zaprezentowany zalążek kodu - jest on jedynie wskazówką i można na swój sposób zaimplementować tę metodę

In [5]:
import torch
from torch import nn, Tensor
import copy
import torch.nn.functional as F

def mlp(encoder_out_shape, projection_size):
    return nn.Sequential(
        nn.Linear(encoder_out_shape, projection_size),
        nn.ReLU(),
        nn.BatchNorm1d(projection_size),
        nn.Linear(projection_size, projection_size)
    )


class SimpleCNNEncoder(torch.nn.Module):
    def __init__(self, repr=80):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Sequential(
            nn.Linear(128*4*4*9, 512),  # Assuming input image size is 32x32
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, repr)
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

class BYOL(nn.Module):
    def __init__(self, projection_size=256, labels_no=1000, tau=0.999):
        super().__init__()
        repr = 128
        self.online_encoder = SimpleCNNEncoder(repr=repr)
        self.online_projector = mlp(repr, projection_size)
        self.online_predictor = mlp(projection_size, labels_no)
        self.online_net = nn.Sequential(
            self.online_encoder,
            self.online_projector,
            self.online_predictor
        )

        self.target_encoder = self.copy_and_freeze_module(self.online_encoder)
        self.target_projector = self.copy_and_freeze_module(mlp(repr, labels_no))

        self.target_net = nn.Sequential(self.target_encoder, self.target_projector)
        for param in self.target_net.parameters():
            param.requires_grad = False

        self.aug_1 = get_default_aug()
        self.aug_2 = get_default_aug()

        self.tau = tau

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        t = self.aug_1(x)
        t_prim = self.aug_2(x)

        q = self.online_net(t)
        q_sym = self.online_net(t_prim)

        with torch.no_grad():
            z_prim = self.target_net(t_prim)
            z_prim_sym = self.target_net(t)

        q = torch.cat([q, q_sym], dim=0)
        z_prim = torch.cat([z_prim, z_prim_sym], dim=0)

        return q, z_prim

    def byol_loss(self, q: Tensor, z_prim: Tensor) -> Tensor:
        q_norm = F.normalize(q, dim=-1, p=2)
        z_prim_norm = F.normalize(z_prim, dim=-1, p=2)

        return 2 - 2 * (q_norm * z_prim_norm).sum(dim=-1).mean()

    @torch.no_grad()
    def update_target_network(self):
        for online_params, target_params in zip(self.online_net.parameters(), self.target_net.parameters()):
            target_params.data = target_params.data * self.tau + online_params.data * (1 - self.tau)


    def copy_and_freeze_module(self, model: nn.Module) -> nn.Module:
        copy_of_model = copy.deepcopy(model)
        for param in copy_of_model.parameters():
            param.requires_grad = False

        return copy_of_model


In [6]:
def fit_ssl(
    model: nn.Module, optimiser: optim.Optimizer,
    train_dl: DataLoader,
    epochs: int,
    print_metrics: bool = False,
    name: str ='',
):
    last_better_epoch = 0
    low_val_loss = float('inf')
    low_acc_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        losses = []
        for X_batch, y_batch in tqdm(train_dl, desc=f"Epoch {epoch}"):
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            q, z = model(X_batch)
            loss = model.byol_loss(q, z)
            losses.append(loss)

            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

        model.eval()
        loss = sum(losses)

        if print_metrics:
            print(
                f"Epoch {name} {epoch}: "
                f"train loss = {loss} "
            )

    return low_val_loss, low_acc_loss

In [7]:
model_ssl = BYOL().cuda()
optimiser = optim.Adam(model_ssl.parameters(), lr=0.001)

result = fit_ssl(model_ssl, optimiser, unlabeled_loader, 10, print_metrics=True, name='byol')

Epoch 0: 100%|██████████| 391/391 [01:28<00:00,  4.42it/s]


Epoch byol 0: train loss = 615.9168701171875 


Epoch 1: 100%|██████████| 391/391 [01:28<00:00,  4.44it/s]


Epoch byol 1: train loss = 601.4818115234375 


Epoch 2: 100%|██████████| 391/391 [01:26<00:00,  4.54it/s]


...

Epoch byol 7: train loss = 594.3836059570312 


Epoch 8: 100%|██████████| 391/391 [01:26<00:00,  4.53it/s]


Epoch byol 8: train loss = 592.0594482421875 


Epoch 9: 100%|██████████| 391/391 [01:25<00:00,  4.55it/s]

Epoch byol 9: train loss = 592.3198852539062 


## Douczanie nadzorowane

***Zaimplementuj*** proces douczania kodera z poprzedniego polecenia na danych etykietowanych ze zbioru treningowego. Porównaj jakość tego modelu z modelem nauczonym tylko na danych etykietownaych. Postaraj się wyjaśnić różnice.

In [8]:
state_dict = model_ssl.online_encoder.state_dict()


In [9]:
encoder = SimpleCNNEncoder(repr=128)
encoder.load_state_dict(state_dict)

class MySSLinear(torch.nn.Module):
  def __init__(self, encoder):
    super().__init__()
    self.encoder = encoder

    self.fc = nn.Sequential(
        nn.Linear(128, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 10),
    )



  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x



In [10]:
model = MySSLinear(encoder).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='ft', patience=3)
print(result)

Epoch 0: 100%|██████████| 157/157 [00:04<00:00, 31.82it/s]


Epoch ft 0: train loss = 0.046 (acc: 0.426), validation loss = 0.047 (acc: 0.414)


Epoch 1: 100%|██████████| 157/157 [00:04<00:00, 35.35it/s]


Epoch ft 1: train loss = 0.040 (acc: 0.551), validation loss = 0.043 (acc: 0.498)


Epoch 2: 100%|██████████| 157/157 [00:04<00:00, 37.03it/s]


...

Epoch ft 6: train loss = 0.010 (acc: 0.907), validation loss = 0.037 (acc: 0.607)


Epoch 7: 100%|██████████| 157/157 [00:03<00:00, 42.29it/s]


Epoch ft 7: train loss = 0.005 (acc: 0.963), validation loss = 0.038 (acc: 0.615)


Epoch 8: 100%|██████████| 157/157 [00:03<00:00, 43.62it/s]


Early stopping on epoch 8
(tensor(0.0339, device='cuda:0'), tensor(0.6145, device='cuda:0'))


In [11]:
model = MySSLinear(encoder).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='ft', patience=3)
print(result)

Epoch 0: 100%|██████████| 157/157 [00:03<00:00, 43.20it/s]


Epoch ft 0: train loss = 0.049 (acc: 0.902), validation loss = 0.057 (acc: 0.611)


Epoch 1: 100%|██████████| 157/157 [00:03<00:00, 45.81it/s]


Epoch ft 1: train loss = 0.044 (acc: 0.951), validation loss = 0.054 (acc: 0.629)


Epoch 2: 100%|██████████| 157/157 [00:03<00:00, 45.28it/s]


...

Epoch ft 6: train loss = 0.033 (acc: 0.995), validation loss = 0.053 (acc: 0.630)


Epoch 7: 100%|██████████| 157/157 [00:03<00:00, 45.01it/s]


Epoch ft 7: train loss = 0.033 (acc: 0.996), validation loss = 0.052 (acc: 0.634)


Epoch 8: 100%|██████████| 157/157 [00:03<00:00, 45.54it/s]


Early stopping on epoch 8
(tensor(0.0515, device='cuda:0'), tensor(0.6396, device='cuda:0'))
